In [1]:
import urllib
import json
import xmltodict
import pandas as pd
import time
import re
from wikiapi import WikiApi

In [2]:
#Useful variables for looping
base_url = "https://en.wikipedia.org/wiki/Deaths_in_" 
months = ["January","February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
years = ["2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016"]

#Useful reference variables
wesco = ["english","welsh","irish","scottish","british", "american", "canadian", "australian","zealand" ] # Western-English speaking country of origin
cpp = ["tv","actor","actress","singer","star","oscar", "nobel","pulitzer","nascar", "television","film","movie","writer","cartoon", "rock", "celebrity", "hockey","football", "baseball", "nfl","nhl", "sports","hollywood"] #celebrity-prone-profession

#Initiate lists of values that will be extracted from Wikipedia
Day = []
Month = []
Year = []
ID = []
RefLength = []
Age = []
WESCO = []
CPP =[]
Other = []

#Loop through years and months to construct the url
for year in years:
    for month in months:
        url = base_url + month + "_" + year
        data = urllib.urlopen(url)
        data = data.read()
        
#Assign appropriate number of days to each month (February will on ocassion have 29 days, I will miss 1 day of values. OK)        
        if month == "January" or month == "March" or month == "May" or month == "July" or month == "August" or month == "October" or month == "December":
            days = range(0, 31)
        elif month == "April" or month == "June" or month == "September" or month == "November":
            days = range(0, 30)
        elif month == "February":
            days = range (0, 28)
        
        for day in days:
#Dig through the XML    
            temp = xmltodict.parse(data)["html"]["body"]["div"][2]["div"][2]["div"][3]["ul"][day]["li"]
            for i in range(len(temp)):
                Day.append(day)
                Month.append(month)
                Year.append(year)

#Extract name and wikipedia entry text length (words)(there will be a slight overestimation because of non-ascii characters)
                try:
                    ref = temp[i]["a"][0]["@href"]
                    try:
                        start = ref.find('/wiki/') + 6
                        found = ref[start:]
                    except:
                        found = "NA"
                    ID.append(found) #name

                    if found != "NA":
                        try:
                            wiki = WikiApi()
                            results = wiki.find(found)
                            article = wiki.get_article(results[0])
                            a = article.content
                            a = a.split()
                            RefLength.append(len(a))
                        except:
                            RefLength.append("NA")
                    else:
                        RefLength.append("NA")
                except:
                    ID.append("NA")
                    RefLength.append("NA")
#Extract age, Western-English speaking country of origin (WESCO), celebrity-prone-profession (CPP)
                try:
                    text =  temp[i]["#text"]
                    Other.append(text)
                    
                    #age
                    a = re.findall(r'\d+',text) #finds all numbers
                    if len(a)==1:
                        Age.append(a)
                    else:
                        Age.append("NA") #if more than one numbers are present I cannot guarantee I can ID the age without further work
                    
                    #wesco & cpp
                    a = [re.findall(x, text.lower())==[x] for x in wesco] #returns list of False/True
                    for i in range(len(a)):
                        test = 0
                        if a[i] == True:
                            test = 1
                            break
                    WESCO.append(test)
                    
                    a = [re.findall(x, text.lower())==[x] for x in cpp]
                    for i in range(len(a)):
                        test = 0
                        if a[i] == True:
                            test = 1
                            break
                    CPP.append(test)
                    
                except:
                    WESCO.append("NA")
                    Age.append("NA")
                    CPP.append("NA")
                    Other.append("NA")
            time.sleep(1)#1 second sleep after each day of the month
    
    temp = pd.DataFrame(
    {'Day': Day,
     'Month': Month,
     'Day': Day,
     'Name': ID,
     'Age': Age,
     'ImportanceIndex': RefLength,
     'WESCO': WESCO,
     'CPP': CPP,
     'Text': Other
    })
    name = year + "_results.csv"
    temp.to_csv(name)
    time.sleep(300) #5minute sleep after each year


/Users/NikiAthanasiadou/anaconda2/envs/basic_env/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Traceback (most recent call last):
  File "/Users/NikiAthanasiadou/anaconda2/envs/basic_env/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1120, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/NikiAthanasiadou/anaconda2/envs/basic_env/lib/python2.7/site-packages/IPython/core/ultratb.py", line 301, in wrapped
    return f(*args, **kwargs)
  File "/Users/NikiAthanasiadou/anaconda2/envs/basic_env/lib/python2.7/site-packages/IPython/core/ultratb.py", line 346, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Users/NikiAthanasiadou/anaconda2/envs/basic_env/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Users/NikiAthanasiadou/anaconda2/envs/basic_env/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/Users/NikiA

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

In [5]:
print ID
print WESCO
print CPP
print Age

[u'Frank_Cary', u'Mapita_Cort%C3%A9s', u'Bryan_Harvey_(musician)', u'Dawn_Lake', u'John_Latham_(artist)', u'Paul_Lindblad', u'Dragan_Luki%C4%87', u'Harry_Magdoff', u'Charles_O._Porter', u'Gideon_Rodan', u'Hubert_Schoemaker', u'Charles_Steen', u'Frank_Butler_(British_sportswriter)', u'Raul_Davila', u'Ofelia_Fox', u'Osa_Massen', u'Klemens_Mielczarek', u'Michael_S._Smith', u'Francis_Steinmetz', u'Frank_Wilkinson', u'John_Wojtowicz', u'John_Woodnutt', u'Lidia_Wysocka', u'Urbano_Lazzaro', u'Steve_Rogers_(rugby_league_footballer)', u'Bill_Skate', u'Bruce_Wilson_(Australian_journalist)', u'John_Hahn-Petersen', u'Sophie_Heathcote', u'Milton_Himmelfarb', u'Stan_Hunt', u'Irving_Layton', u'Maktoum_bin_Rashid_Al_Maktoum', u'Gretl_Sch%C3%B6rg', u'Lourdes_Van-D%C3%BAnem', u'Nel_van_Vliet', u'Ramona_Bell', u'Rod_Dedeaux', u'Keizo_Miura', u'Merlyn_Rees', u'Ken_Mosdell', u'Mark_Roberts_(actor)', u'Simon_Shanks', u'Rachel_Squire', u'Alex_St._Clair', u'Vajramuni', u'Allaire_du_Pont', u'Roshan_Khan', u'J%

In [ ]:
data1 = pd.DataFrame( columns= ["#text", "a", "Year", "Month", "Day"])

base_url = "https://en.wikipedia.org/wiki/Deaths_in_" 

months = ["January","February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
years = ["2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016"]

for year in years:
    for month in months:
        url = base_url + month + "_" + year
        data = urllib.urlopen(url)
        data = data.read()
        
        if month == "January" or month == "March" or month == "May" or month == "July" or month == "August" or month == "October" or month == "December":
            days = range(0, 31)
        elif month == "April" or month == "June" or month == "September" or month == "November":
            days = range(0, 30)
        elif month == "February":
            days = range (0, 28)
        
        for day in days:
            temp = xmltodict.parse(data)["html"]["body"]["div"][2]["div"][2]["div"][3]["ul"][day]["li"]

            for i in range(len(temp)):
                print  i
                try:
                    print temp[i]["a"][0]["@href"]
                except:
                    print "zong! @href"
                try:
                    print temp[i]["#text"]
                except:
                    print "zong! #text"
            time.sleep(1)



